In [7]:
#HACER UNO PARA CADA TEMPLATE
#HACER UNO PARA CADA COLEGIO
#HACER UN REPORTE EN EXCEL Y OTRO EN WORD
#HACER UN FOLDER PARA CADA REPORTE
#HACER UNO PARA IMAGENES



import openpyxl
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, unquote
from tqdm.notebook import tqdm
import csv

import tkinter as tk
from tkinter import filedialog
from IPython.display import display
import os
import openpyxl
from tkinter import messagebox
from threading import Thread
from tkinter.ttk import Progressbar


#Obtain the final url from the relative url
def get_final_url(url):
    try:
        response = requests.get(url, allow_redirects=True)
        return response.url
    except requests.exceptions.RequestException:
        return None

#Clean the link, don't check mailto or anchors
def clean_link(link):
    if link and not link.startswith("mailto:") and not link.startswith("#"):
        link = unquote(link).strip()
        #CHANGED TO WHILE LOOP
        while link.endswith("%20"):
            link = link[:-1]
        #if link.endswith("%20"):
        #    link = link[:-3]  # Remove trailing %20 if it exists
        return link
    return None

#get all the links from page
def get_links_from_page(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            links = [clean_link(link.get('href')) for link in soup.find_all('a', href=True)]
            return [link for link in links if link is not None]  # Filter out None values
        else:
            return []
    except requests.exceptions.RequestException:
        return []

#Identify whether the link is on navigation, footer, or main content (Add left navigation)
def identify_section(link, page_link):
    try:
        absolute_page_link = urljoin(link, page_link)
        response = requests.get(link)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            # Search for partial matches of the href attribute with page_link
            element = soup.find('a', href=lambda href: page_link in href)
            if element:
                # Search for the outer elements to determine the section
                #change to id siteheader
                nav_element = element.find_parent('nav')
                if nav_element:
                    return "Navigation"
                
                footer_element = element.find_parent('footer')
                if footer_element:
                    return "Footer"
                
                sidr_container_element = element.find_parent('div', id='sidr-container')
                if sidr_container_element:
                    return "Main Content"
                
            #BELOW SHOULDN'T BE USED, JUST IN CASE
            return "Unknown"  # If the element is not found in any of the relevant outer elements
        else:
            return "Unknown"  # If there was an issue fetching the page
    except requests.exceptions.RequestException:
        return "Unknown"  # If there was a connection error or other issues


#Main function to check for broken links given an excel
def check_broken_links(excel_file):
    try:
        print(f"Opening file: {excel_file}...")
        wb = openpyxl.load_workbook(excel_file)
        sheet = wb.active
        column_with_links = 'A'

        broken_links_report = []
        total_links_checked = 0
        total_links = 0

        for cell in sheet[column_with_links]:
            if cell.value:
                total_links += 1

        print(f"Checking {total_links} links inside {excel_file}...")
        for cell in tqdm(sheet[column_with_links], total=total_links, desc="Progress"):
            if cell.value:
                link = cell.value
                total_links_checked += 1
                page_links = get_links_from_page(link)

                # Page progress bar starts here
                page_progress_desc = f"Page Progress ({total_links_checked}/{total_links})"
                with tqdm(total=len(page_links), desc=page_progress_desc, leave=False) as page_progress:
                    for page_link in page_links:
                        absolute_link = urljoin(link, page_link)
                        final_url = get_final_url(absolute_link)
                        section = identify_section(link, page_link)  # Pass both link and page_link
                        if final_url is not None:
                            try:
                                response = requests.get(final_url)
                                if response.status_code != 200:
                                    
                                    #MIGHT ERASE THIS LINE TO ALLOW DUPLICATES
                                    if (link, final_url, section, page_link) not in broken_links_report:  # Check for duplicates
                                        broken_links_report.append((link, final_url, section, page_link))
                                        tqdm.write(f"Broken link found on page '{link}' in '{section}' section: {final_url}", end='\r')
                                    else:
                                        tqdm.write(f"Duplicate broken link found on page '{link}': {final_url}", end='\r')
                                else:
                                    pass  # No need for this message
                            except requests.exceptions.RequestException:
                                if (link, final_url, section, page_link) not in broken_links_report:  # Check for duplicates
                                    broken_links_report.append((link, final_url, section, page_link))
                                    tqdm.write(f"Connection error occurred for link on page '{link}' in '{section}' section: {final_url}", end='\r')
                                else:
                                    tqdm.write(f"Duplicate connection error occurred for link on page '{link}': {final_url}", end='\r')
                        else:
                            # Include invalid links in the broken links report
                            broken_links_report.append((link, page_link, section, "Invalid URL"))
                            tqdm.write(f"Invalid URL: {absolute_link}", end='\r')
                        page_progress.update(1)

        print(f"\nTotal pages checked: {total_links_checked}")
        print(f"Total broken links found: {len(broken_links_report)}")
        return broken_links_report
    except openpyxl.utils.exceptions.InvalidFileException:
        print(f"Error: Unable to open the file '{excel_file}'. Please check if it's a valid Excel file.")
        return []
    except Exception as e:
        print(f"Error: An unexpected error occurred while processing the file '{excel_file}': {e}")
        return []


def save_to_csv(file_path, data, header):
    with open(file_path, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(header)
        csv_writer.writerows(data)

if __name__ == "__main__":
    excel_file_path = "example.xlsx"
    broken_links_report = check_broken_links(excel_file_path)

    if broken_links_report:
        print("Broken Links Report:")
        for link, page_link, section, relative in broken_links_report:
            print(f"Broken Link on page '{link}' in '{section}' section: {page_link}, relative: {relative}")

        # Save broken links report to CSV
        csv_file_path = "broken_links_report.csv"
        header = ["Page", "Broken Link", "Section"]
        save_to_csv(csv_file_path, broken_links_report, header)
        print(f"Broken links report saved to '{csv_file_path}'")
    else:
        print("No broken links found.")


Opening file: example.xlsx...
Checking 3 links inside example.xlsx...


Progress:   0%|          | 0/3 [00:00<?, ?it/s]

Page Progress (1/3):   0%|          | 0/57 [00:00<?, ?it/s]

Page Progress (2/3):   0%|          | 0/115 [00:00<?, ?it/s]

Page Progress (3/3):   0%|          | 0/128 [00:00<?, ?it/s]

Invalid URL: http://sao.fraud.state.tx.us/p.edu/engineering/student-resources/student-resources.html' in 'Navigation' section: https://www.utep.edu/engineering/student-resources/student-resources-tutoring.html-faqs.htmlml
Total pages checked: 3
Total broken links found: 29
Broken Links Report:
Broken Link on page 'https://utep.edu/' in 'Unknown' section: https://utepminers.com/?utep-home, relative: https://utepminers.com/?utep-home
Broken Link on page 'https://utep.edu/' in 'Unknown' section: https://www.linkedin.com/school/the-university-of-texas-at-el-paso/, relative: https://www.linkedin.com/school/the-university-of-texas-at-el-paso/
Broken Link on page 'https://utep.edu/' in 'Unknown' section: http://sao.fraud.state.tx.us/?utep-home, relative: Invalid URL
Broken Link on page 'https://www.utep.edu/engineering/student-resources/feedback.html' in 'Navigation' section: https://www.utep.edu/engineering/student-resources/, relative: /engineering/student-resources
Broken Link on page 'htt

In [ ]:


#HACER UNO PARA CADA TEMPLATE
#HACER UNO PARA CADA COLEGIO
#HACER UN REPORTE EN EXCEL Y OTRO EN WORD
#HACER UN FOLDER PARA CADA REPORTE
#HACER UNO PARA IMAGENES



import openpyxl
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, unquote
from tqdm.notebook import tqdm
import csv

import tkinter as tk
from tkinter import filedialog
from IPython.display import display
import os
import openpyxl
from tkinter import messagebox
from threading import Thread
from tkinter.ttk import Progressbar


#Obtain the final url from the relative url
def get_final_url(url):
    try:
        response = requests.get(url, allow_redirects=True)
        return response.url
    except requests.exceptions.RequestException:
        return None

#Clean the link, don't check mailto or anchors
def clean_link(link):
    if link and not link.startswith("mailto:") and not link.startswith("#"):
        link = unquote(link).strip()
        #CHANGED TO WHILE LOOP
        while link.endswith("%20"):
            link = link[:-1]
        #if link.endswith("%20"):
        #    link = link[:-3]  # Remove trailing %20 if it exists
        return link
    return None

#get all the links from page
def get_links_from_page(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            links = [clean_link(link.get('href')) for link in soup.find_all('a', href=True)]
            return [link for link in links if link is not None]  # Filter out None values
        else:
            return []
    except requests.exceptions.RequestException:
        return []

#Identify whether the link is on navigation, footer, or main content (Add left navigation)
def identify_section(link, page_link):
    try:
        absolute_page_link = urljoin(link, page_link)
        response = requests.get(link)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            # Search for partial matches of the href attribute with page_link
            element = soup.find('a', href=lambda href: page_link in href)
            if element:
                # Search for the outer elements to determine the section
                #change to id siteheader
                nav_element = element.find_parent('nav')
                if nav_element:
                    return "Navigation"
                
                footer_element = element.find_parent('footer')
                if footer_element:
                    return "Footer"
                
                sidr_container_element = element.find_parent('div', id='sidr-container')
                if sidr_container_element:
                    return "Main Content"
                
            #BELOW SHOULDN'T BE USED, JUST IN CASE
            return "Unknown"  # If the element is not found in any of the relevant outer elements
        else:
            return "Unknown"  # If there was an issue fetching the page
    except requests.exceptions.RequestException:
        return "Unknown"  # If there was a connection error or other issues


#Main function to check for broken links given an excel
def check_broken_links(excel_file):
    try:
        print(f"Opening file: {excel_file}...")
        wb = openpyxl.load_workbook(excel_file)
        sheet = wb.active
        column_with_links = 'A'

        broken_links_report = []
        total_links_checked = 0
        total_links = 0

        for cell in sheet[column_with_links]:
            if cell.value:
                total_links += 1

        print(f"Checking {total_links} links inside {excel_file}...")
        for cell in tqdm(sheet[column_with_links], total=total_links, desc="Progress"):
            if cell.value:
                link = cell.value
                total_links_checked += 1
                page_links = get_links_from_page(link)

                # Page progress bar starts here
                page_progress_desc = f"Page Progress ({total_links_checked}/{total_links})"
                with tqdm(total=len(page_links), desc=page_progress_desc, leave=False) as page_progress:
                    for page_link in page_links:
                        absolute_link = urljoin(link, page_link)
                        final_url = get_final_url(absolute_link)
                        section = identify_section(link, page_link)  # Pass both link and page_link
                        if final_url is not None:
                            try:
                                response = requests.get(final_url)
                                if response.status_code != 200:
                                    
                                    #MIGHT ERASE THIS LINE TO ALLOW DUPLICATES
                                    if (link, final_url, section, page_link) not in broken_links_report:  # Check for duplicates
                                        broken_links_report.append((link, final_url, section, page_link))
                                        tqdm.write(f"Broken link found on page '{link}' in '{section}' section: {final_url}", end='\r')
                                    else:
                                        tqdm.write(f"Duplicate broken link found on page '{link}': {final_url}", end='\r')
                                else:
                                    pass  # No need for this message
                            except requests.exceptions.RequestException:
                                if (link, final_url, section, page_link) not in broken_links_report:  # Check for duplicates
                                    broken_links_report.append((link, final_url, section, page_link))
                                    tqdm.write(f"Connection error occurred for link on page '{link}' in '{section}' section: {final_url}", end='\r')
                                else:
                                    tqdm.write(f"Duplicate connection error occurred for link on page '{link}': {final_url}", end='\r')
                        else:
                            # Include invalid links in the broken links report
                            broken_links_report.append((link, page_link, section, "Invalid URL"))
                            tqdm.write(f"Invalid URL: {absolute_link}", end='\r')
                        page_progress.update(1)

        print(f"\nTotal pages checked: {total_links_checked}")
        print(f"Total broken links found: {len(broken_links_report)}")
        return broken_links_report
    except openpyxl.utils.exceptions.InvalidFileException:
        print(f"Error: Unable to open the file '{excel_file}'. Please check if it's a valid Excel file.")
        return []
    except Exception as e:
        print(f"Error: An unexpected error occurred while processing the file '{excel_file}': {e}")
        return []


def save_to_csv(file_path, data, header):
    with open(file_path, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(header)
        csv_writer.writerows(data)





def browse_folder():
    folder_path = filedialog.askdirectory()
    folder_var.set(folder_path)
    update_excel_list()

def update_excel_list():
    folder_path = folder_var.get()
    excel_files = [f for f in os.listdir(folder_path) if f.endswith(".xlsx")]
    excel_listbox.delete(0, tk.END)
    for excel_file in excel_files:
        excel_listbox.insert(tk.END, excel_file)

def start_link_checking():
    folder_path = folder_var.get()
    selected_excel = excel_listbox.get(tk.ACTIVE)
    
    if not selected_excel:
        messagebox.showinfo("Error", "Please select an Excel file.")
        return

    excel_file_path = os.path.join(folder_path, selected_excel)
    broken_links_report = check_broken_links(excel_file_path)

    # Display the results or save to CSV or any other actions you want to take with the report

def check_links_thread():
    progress_bar.start()
    start_link_checking()
    progress_bar.stop()

# Tkinter GUI
root = tk.Tk()
root.title("Broken Link Checker")

folder_var = tk.StringVar()

folder_label = tk.Label(root, text="Select Folder:")
folder_label.pack()

folder_frame = tk.Frame(root)
folder_frame.pack()

folder_entry = tk.Entry(folder_frame, textvariable=folder_var)
folder_entry.pack(side=tk.LEFT, padx=5)

folder_browse_button = tk.Button(folder_frame, text="Browse", command=browse_folder)
folder_browse_button.pack(side=tk.LEFT)

excel_list_label = tk.Label(root, text="Excel Files in Selected Folder:")
excel_list_label.pack()

excel_listbox = tk.Listbox(root, selectmode=tk.SINGLE, exportselection=False)
excel_listbox.pack()

progress_bar = Progressbar(root, mode='indeterminate')

start_button = tk.Button(root, text="Check for broken links", command=lambda: Thread(target=check_links_thread).start())
start_button.pack()

# Display the GUI in the Jupyter Notebook
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Ricar\anaconda3\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\Ricar\AppData\Local\Temp\ipykernel_18716\261271030.py", line 170, in browse_folder
    update_excel_list()
  File "C:\Users\Ricar\AppData\Local\Temp\ipykernel_18716\3344881128.py", line 175, in update_excel_list
    excel_listbox.delete(0, tk.END)
  File "C:\Users\Ricar\anaconda3\lib\tkinter\__init__.py", line 3218, in delete
    self.tk.call(self._w, 'delete', first, last)
_tkinter.TclError: invalid command name ".!listbox"
